### We can construct a mosaic of nearby tiles using this method: https://www.kaggle.com/c/planet-understanding-the-amazon-from-space/discussion/36738

# Preprocess data
---

In [32]:
# extract labels from training data
import pandas as pd

train_data = pd.read_csv('data/train_v2.csv/train_v2.csv')
print(train_data['tags'].values)

unique_labels = []
for line in train_data['tags'].values:
    for label in line.split():
        if label not in unique_labels:
            unique_labels.append(label)
            print(label)

print(len(unique_labels))

['haze primary' 'agriculture clear primary water' 'clear primary' ...
 'agriculture clear primary' 'agriculture clear primary road'
 'agriculture cultivation partly_cloudy primary']
haze
primary
agriculture
clear
water
habitation
road
cultivation
slash_burn
cloudy
partly_cloudy
conventional_mine
bare_ground
artisinal_mine
blooming
selective_logging
blow_down
17


# Construct model
---

# Train model
---

# View results
---